In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 921.5/921.5 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 59.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 58.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [ ]:
import cv2
import numpy as np
import os
import shutil
import matplotlib.pyplot as plt
from ultralytics import YOLO
from sklearn.cluster import KMeans
from google.colab import files
from google.colab import drive

drive.mount('/content/drive')

output_folder = "/content/drive/My Drive/Image_ND/frames"
os.makedirs(output_folder, exist_ok=True)

# 1. Vaizdo įrašo suskaidymas į kadrus
def extract_frames(video_path, output_folder):
    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame_path = os.path.join(output_folder, f"frame_{frame_count:04d}.jpg")
        cv2.imwrite(frame_path, frame)
        frame_count += 1
    cap.release()
    return frame_count

# 2. Objekto aptikimas YOLOv8 pagalba
model = YOLO("yolov8n.pt")

def detect_objects(image_path):
    image = cv2.imread(image_path)
    results = model(image)
    return results[0]

def analyze_frames(output_folder):
    scores = []
    frame_files = sorted([f for f in os.listdir(output_folder) if f.endswith(".jpg")])

    for frame in frame_files:
        frame_path = os.path.join(output_folder, frame)
        result = detect_objects(frame_path)
        confidences = [box[4].item() for box in result.boxes.data]  # Gauta pasitikėjimo reikšmė
        avg_confidence = np.mean(confidences) if confidences else 0  # Jei nėra objektų, tikimybė 0
        scores.append((frame_path, avg_confidence, result))

    return scores

# 3. Klasterizavimas pagal aptikimo kokybę
def cluster_frames(scores, n_clusters=3):
    frame_paths, conf_values, _ = zip(*scores)
    conf_values = np.array(conf_values).reshape(-1, 1)

    kmeans = KMeans(n_clusters=n_clusters, random_state=0, n_init=10)
    clusters = kmeans.fit_predict(conf_values)

    clustered_frames = {}
    for i in range(n_clusters):
        clustered_frames[f"Cluster_{i}_Conf_{kmeans.cluster_centers_[i][0]:.2f}"] = []

    for frame, cluster in zip(frame_paths, clusters):
        clustered_frames[f"Cluster_{cluster}_Conf_{kmeans.cluster_centers_[cluster][0]:.2f}"].append(frame)

    return clustered_frames

# 4. Rezultatų vizualizacija su grafikais
def plot_results(scores, output_folder):
    for frame_path, confidence, result in scores:
        image = cv2.imread(frame_path)
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Apibraukiame aptiktus objektus
        for box in result.boxes.data:
            x1, y1, x2, y2, conf, cls = box.tolist()
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            label = f"{result.names[int(cls)]}: {conf:.2f}"
            cv2.rectangle(image_rgb, (x1, y1), (x2, y2), (255, 0, 0), 2)
            cv2.putText(image_rgb, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

        fig, ax = plt.subplots(1, 2, figsize=(10, 5))

        # Rodome kadrą
        ax[0].imshow(image_rgb)
        ax[0].set_title("Kadras su aptiktais objektais")
        ax[0].axis("off")

        # Rodome tikimybių grafiką
        ax[1].barh(["Confidence"], [confidence], color='green')
        ax[1].set_xlim(0, 1)
        ax[1].set_title("Aptikimo pasitikėjimas")

        save_path = os.path.join(output_folder, os.path.basename(frame_path).replace(".jpg", "_analysis.jpg"))
        plt.savefig(save_path)
        plt.close()

def identify_low_confidence_frames(scores, threshold=0.3):
    low_confidence_frames = [frame for frame, conf, _ in scores if conf < threshold]
    return low_confidence_frames

# Failo įkėlimas rankiniu būdu
uploaded = files.upload()
video_path = list(uploaded.keys())[0]

# Vykdome analizę
total_frames = extract_frames(video_path, output_folder)
scores = analyze_frames(output_folder)
plot_results(scores, output_folder)

# Klasterizavimas ir mažo pasitikėjimo kadrų identifikavimas
clustered_frames = cluster_frames(scores)
low_confidence_frames = identify_low_confidence_frames(scores)

print("Mažo pasitikėjimo kadrai:", low_confidence_frames)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Saving path_to_video.mp4 to path_to_video (1).mp4

0: 384x640 9 persons, 1 bicycle, 5 cars, 1 motorcycle, 130.2ms
Speed: 5.2ms preprocess, 130.2ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 persons, 1 bicycle, 5 cars, 1 motorcycle, 1 traffic light, 175.7ms
Speed: 5.7ms preprocess, 175.7ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 persons, 1 bicycle, 3 cars, 1 motorcycle, 1 handbag, 106.8ms
Speed: 4.6ms preprocess, 106.8ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 persons, 1 bicycle, 3 cars, 1 motorcycle, 2 handbags, 114.7ms
Speed: 3.6ms preprocess, 114.7ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 persons, 1 bicycle, 3 cars, 1 motorcycle, 1 handbag, 104.0ms
Speed: 3.5ms preprocess, 104.0ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 persons, 1 bicycle, 3 cars, 1 motorcycle, 1 handbag, 108.7ms
Speed: 3.6m